<a href="https://colab.research.google.com/github/Yissan8/IPR/blob/main/IPR_test_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BUILDING BLOCKS**

In [2]:
import numpy as np #basic libraries import
import pandas as pd
import datetime as dt
from urllib.request import urlretrieve
import warnings

warnings.filterwarnings('ignore') #disable libraries warning due to legacy functions


positions, MW = pd.read_excel("/content/IPR Role - Test Case.xlsx",sheet_name=['clientPositions','modelWeights']).values() # import our two original datasets thefrom excel shit

priceDB  = positions[positions['implementationType']=='segregated'][['asOfDate','accountId','fmcSecurityId', 'securityName', 'priceCad']] #create a separate database of prices from the posistions dataset : reason behind this is hat we might end up needing to trade(sell or buy) a security for a specific model/account that is not currently held in the said account, but held under a different account/model from which we could obtain the current market price for the security

class blotter: #blotter datatype
  def __init__(self, df):
    self.TradeDate = df.iloc[:,0]
    self.SettlementDate = df.iloc[:,1]
    self.Side = df.iloc[:,2]
    self.AccountID = df.iloc[:,3]
    self.frmSecurityId = df.iloc[:,4]
    self.Quantity = df.iloc[:,5]
    self.marketValueCad = df.iloc[:,6]

  def show(self)-> pd.DataFrame:
    return pd.DataFrame({'TradeDate':self.TradeDate,"SettlementDate":self.SettlementDate,'Side':self.Side,'AccountID':self.AccountID,"frmSecurityId":self.frmSecurityId,"Quantity":self.Quantity,'marketValueCad':self.marketValueCad})


#**Rebalancing function for CASE 1**

We take as input the account, the model dataset, the positions dataset, the prices, and the date for which we would like to generate the rebalancing trades

In [ ]:
def rebalance(account:str ,model: pd.DataFrame,CP:pd.DataFrame,prices:pd.DataFrame,date=pd.Timestamp('2024-07-12 00:00:00'))->blotter:

  #couple of user-proofing exception handlers
  #first two tries are to make sure the positions input dataframe is properly filtered to only keep the segregated funds
  try:
    CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segregated') & (CP['asOfDate']==date)]
  except:
      try:
        CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segragated')]
      except:
        CP = CP[CP['accountId']==account]


  CP['account_AUM'] = CP.groupby(['asOfDate','accountId'])['marketValueCad'].transform('sum') # AUM for the desired account and date
  CP['currentweights'] = CP['marketValueCad']/CP['account_AUM'] #current weights of each security in the desired account and date

  #merge with the model dataset and the prices dataset:
  #we perform a full outer join to ensure to keep all the positions, because the current account might either be lacking some stocks from the model or be holding stocks not prescribed by the model.
  #For the former case we will need to buy, for the latter we will need to liquidate. The outer join ensures we keep all the stocks from both sides

  DS = CP.merge(model[['asOfDate', 'fmcSecurityId', 'modelId','modelWeight']],left_on = ['asOfDate', 'fmcSecurityId', 'modelId'],right_on = ['asOfDate', 'fmcSecurityId', 'modelId'],how='outer').merge(prices,left_on=['asOfDate', 'fmcSecurityId','accountId'],right_on=['asOfDate', 'fmcSecurityId','accountId'])

  #because of th outer join, we might end up with multiple NaNs values for non held or held but not prescribed stocks. here we perform the equivalent of an SQL "ISNULL" by filling the NaNs with relevant values
  DS['quantity'], DS['currentweights'],DS['account_AUM'] = DS['quantity'].fillna(0), DS['currentweights'].fillna(0),DS['account_AUM'].fillna(DS['account_AUM'].iloc[0])


  #Now we can peacefully compute the transactions to be instructed as well as their marketValues
  DS['transaction'], DS['newMV'] = round(DS['modelWeight'] * DS['account_AUM']/DS['priceCad']) - DS['quantity'], (round(DS['modelWeight'] * DS['account_AUM']/DS['priceCad']) - DS['quantity'])*DS['priceCad']

  #Transaction side and settlement date for blotter object ; Settlement date is Trade date + 1 business day since 2024/05/29 in Canada and the US
  DS['Side'], DS['SettlementDate'] = DS.apply(lambda row: 'A' if row['transaction'] == - row['quantity'] else "B" if row['transaction'] >  row['quantity'] else "S",axis=1), DS['asOfDate'] + pd.offsets.BusinessDay(n=1)

  return blotter(DS[['asOfDate','SettlementDate','Side', 'accountId','fmcSecurityId','transaction','newMV']])

#AUM change function for case 2

We take the same inputs as the previous function, with an important addition : The rebal argument : It lets us choose whether we want to simply keep the same currently held model weights when changing AUM ("NO"),
or if on top of changing the account AUM we also want to perform a rebalancing to bring the account back on the model prescribed weights("YES")

In [ ]:
def AUM(account:str,CP: pd.DataFrame,model: pd.DataFrame,prices:pd.DataFrame,amount, date=pd.Timestamp('2024-07-12 00:00:00'),rebal:str = "NO"):

    #exception handling for our users that are too creative
    try:
      ['YES','NO'].count(rebal)==0

      try:
        CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segregated') & (CP['asOfDate']==date)]
      except:
        try:
          CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segragated')]
        except:
            CP = CP[CP['accountId']==account]


      CP['account_AUM'] = CP.groupby(['asOfDate','accountId'])['marketValueCad'].transform('sum') #AUM compute as before
      CP['currentweights'] = CP['marketValueCad']/CP['account_AUM'] #current weights as before

      #modelweights and prices addition through join operation. We add the modelweights in case rebal argument is YES, otherwise we don't need it
      DS = CP.merge(model[['asOfDate', 'fmcSecurityId', 'modelId','modelWeight']],left_on = ['asOfDate', 'fmcSecurityId', 'modelId'],right_on = ['asOfDate', 'fmcSecurityId', 'modelId'],how='outer').merge(prices,left_on=['asOfDate', 'fmcSecurityId','accountId'],right_on=['asOfDate', 'fmcSecurityId','accountId'])

      #fillna because of outer join
      DS['quantity'], DS['currentweights'],DS['account_AUM'] = DS['quantity'].fillna(0), DS['currentweights'].fillna(0),DS['account_AUM'].fillna(DS['account_AUM'].iloc[0])

      #find the transactions to instruct
      if rebal=='NO':
        DS['transaction'] =  round(DS['currentweights'] * (DS['account_AUM']+amount)/DS['priceCad'])  - DS['quantity']

      elif rebal =='YES':
        DS['transaction'] = round(DS['modelWeight'] * (DS['account_AUM']+amount)/DS['priceCad']) - DS['quantity']

      #Side, Settlement Date and transaction marketvalue
      DS['Side'], DS['SettlementDate'], DS['newMV'] = DS.apply(lambda row: 'A' if row['transaction'] == - row['quantity'] else "B" if row['transaction'] >  0 else "S",axis=1), DS['asOfDate'] + pd.offsets.BusinessDay(n=1),DS['transaction']*DS['priceCad']

      return blotter(DS[['asOfDate','SettlementDate','Side', 'accountId','fmcSecurityId','transaction','newMV']])

    except:
      print("Please select a valid input for rebal argument, either YES or NO")


#AUM Change - extended

Here we try to extend the previous function scope to get a more realistic result by simulating actual trades execution constraints : bid-ask spread and transaction costs
We implement a very naive version of those two constraints y adding the following two variables :
- **bid_ask** is a list variable that allows us to specify a percentage range around the provide priceCad of the stock to simulate a bid-ask spread .For example, if the provided price is 100 and bid_ask is **[-0.1, 0.1]**, we will assume that the bid-ask for this security is **90-110**
- **TC** is to simulate transactions costs, is a percentage of the traded market value that is paid as transaction costs.
- We keep the provied priceCad value as 'mid' price to compute marketvalue
- Ideally we would also have variable for tax constraints, liquidity, market impact, etc.
- The below function is almost identical to the previous one, except or the trade execution prices

In [3]:
def AUM_ex(account:str,CP: pd.DataFrame,model: pd.DataFrame,prices:pd.DataFrame,amount,rebal:str = "NO", date=pd.Timestamp('2024-07-12 00:00:00'),bid_ask:list = [0,0],TC:float = 0):

    try:
      ['YES','NO'].count(rebal)==0
      try:
        CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segregated') & (CP['asOfDate']==date)]
      except:
        try:
          CP = CP[(CP['accountId']==account) & (CP['implementationType']=='segragated')]
        except:
            CP = CP[CP['accountId']==account]



      CP['account_AUM'] = CP.groupby(['asOfDate','accountId'])['marketValueCad'].transform('sum')
      CP['currentweights'] = CP['marketValueCad']/CP['account_AUM']

      DS = CP.merge(model[['asOfDate', 'fmcSecurityId', 'modelId','modelWeight']],left_on = ['asOfDate', 'fmcSecurityId', 'modelId'],right_on = ['asOfDate', 'fmcSecurityId', 'modelId'],how='outer').merge(prices,left_on=['asOfDate', 'fmcSecurityId','accountId'],right_on=['asOfDate', 'fmcSecurityId','accountId'])
      DS['quantity'], DS['currentweights'],DS['account_AUM'] = DS['quantity'].fillna(0), DS['currentweights'].fillna(0),DS['account_AUM'].fillna(DS['account_AUM'].iloc[0])
      DS['BID'], DS['ASK'] = DS['priceCad']*(1+bid_ask[0]), DS['priceCad']*(1+bid_ask[1])

      if rebal=='NO':
        DS['theoritical_transaction'] =  round(DS['currentweights'] * (DS['account_AUM']+amount)/DS['priceCad'])  - DS['quantity']

      if rebal =='YES':
        DS['theoritical_transaction'] = round(DS['modelWeight'] * (DS['account_AUM']+amount)/DS['priceCad']) - DS['quantity']

      DS['Side'] = DS.apply(lambda row: 'A' if row['theoritical_transaction'] == - row['quantity'] else "B" if row['theoritical_transaction'] >  0 else "S",axis=1)
      DS['transaction'] = pd.DataFrame([ round(DS['currentweights'][i] * (DS['account_AUM'][i]+amount)*(1-TC)/DS['BID'][i]) - DS['quantity'][i] if DS['Side'][i]=='S' else round(DS['currentweights'][i] * (DS['account_AUM'][i]+amount)*(1+TC)/DS['ASK'][i]) - DS['quantity'][i] for i in range(len(DS['theoritical_transaction'])) ])
      DS['newMV'] = DS['transaction']*DS['priceCad']
      DS['SettlementDate'] =  DS['asOfDate'] + pd.offsets.BusinessDay(n=1)

      return blotter(DS[['asOfDate','SettlementDate','Side', 'accountId','fmcSecurityId','transaction','newMV']])

    except:
      print("Please select a valid input for rebal argument, either YES or NO")

# CASE 1

In [ ]:
rebalancing_trades = rebalance("A5",MW,positions.drop(['priceCad','securityName'],axis=1),priceDB)
rebalancing_trades.show()

,TradeDate,SettlementDate,Side,AccountID,frmSecurityId,Quantity,marketValueCad
0,2024-07-12,2024-07-15,S,A5,02079K30 F,5.0,9542.346655
1,2024-07-12,2024-07-15,S,A5,03265410 F,7.0,1172.323129
2,2024-07-12,2024-07-15,S,A5,05333210 A,7.0,10458.385840
3,2024-07-12,2024-07-15,S,A5,07588710 A,5.0,1620.899006
4,2024-07-12,2024-07-15,B,A5,14448C10 A,470.0,13957.854519
5,2024-07-12,2024-07-15,S,A5,12572Q10 A,20.0,4414.768453
6,2024-07-12,2024-07-15,S,A5,19416210 A,58.0,5778.618281
7,2024-07-12,2024-07-15,S,A5,27886510 A,-21.0,-5702.033773
8,2024-07-12,2024-07-15,S,A5,30307510 A,10.0,4461.849384
9,2024-07-12,2024-07-15,S,A5,38410910 A,-252.0,-16462.144755


# CASE 2

In [ ]:
increase_trades = AUM("B5",positions.drop(['priceCad','securityName'],axis=1),MW,priceDB,amount=1000000)
increase_trades.show()

,TradeDate,SettlementDate,Side,AccountID,frmSecurityId,Quantity,marketValueCad
0,2024-07-12,2024-07-15,B,B5,02079K30 F,27.0,52357.485513
1,2024-07-12,2024-07-15,B,B5,05333210 A,20.0,30799.973309
2,2024-07-12,2024-07-15,B,B5,07588710 A,104.0,33858.566040
3,2024-07-12,2024-07-15,B,B5,14448C10 A,234.0,7109.308904
4,2024-07-12,2024-07-15,B,B5,12572Q10 A,161.0,35653.912054
5,2024-07-12,2024-07-15,B,B5,023740 X,501.0,22603.875112
6,2024-07-12,2024-07-15,B,B5,635693 X,124.0,30125.158871
7,2024-07-12,2024-07-15,B,B5,B1WGG9 X,41.0,27962.844746
8,2024-07-12,2024-07-15,B,B5,38410910 A,366.0,23926.501411
9,2024-07-12,2024-07-15,B,B5,40415F10 A,305.0,18884.756744


 # CASE 2 - With Bid-Ask Spread and Transaction costs

In [4]:
increase_trades_real = AUM_ex("B5",positions.drop(['priceCad','securityName'],axis=1),MW,priceDB,amount=1000000,rebal='NO',bid_ask=[-0.01,0.01],TC=0.001)
increase_trades_real.show()

,TradeDate,SettlementDate,Side,AccountID,frmSecurityId,Quantity,marketValueCad
0,2024-07-12,2024-07-15,B,B5,02079K30 F,26.0,50418.319383
1,2024-07-12,2024-07-15,B,B5,05333210 A,19.0,29259.974643
2,2024-07-12,2024-07-15,B,B5,07588710 A,99.0,32230.750365
3,2024-07-12,2024-07-15,B,B5,14448C10 A,223.0,6775.110623
4,2024-07-12,2024-07-15,B,B5,12572Q10 A,153.0,33882.289095
5,2024-07-12,2024-07-15,B,B5,023740 X,478.0,21566.172263
6,2024-07-12,2024-07-15,B,B5,635693 X,118.0,28667.489893
7,2024-07-12,2024-07-15,B,B5,B1WGG9 X,39.0,26598.803539
8,2024-07-12,2024-07-15,B,B5,38410910 A,349.0,22815.161181
9,2024-07-12,2024-07-15,B,B5,40415F10 A,291.0,18017.915451
